# Traffic Sign Classification Challenge 2022
Author: Agnieszka Mikołajczyk


Now, go to your kaggle account and download dataset. Upload it below.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip -q traffic-sign-2022.zip #uzipping dataset

Now we will seed everything so the results are reproducible

In [ ]:
import random
import numpy as np
import torch
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(2021)

## Loading data
Now, we will preprocess and load training set

In [ ]:
NUM_EPOCHS = 20 # number of times which the entire dataset is passed throughout the model
BATCH_SIZE = 64 # the size of input data took for one iteration
lr = 1e-3 # size of step
INPUT_SIZE = 28

In [ ]:
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

transform = transforms.Compose([
        transforms.Resize([INPUT_SIZE,INPUT_SIZE]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4588, 0.5020, 0.5196],
                             std=[0.1665, 0.1690, 0.1644])
    ])

train_dataset = dsets.ImageFolder(
        root='data/train',
        transform=transform,
    )

train_size = round(len(train_dataset) * 0.6)
valid_size = len(train_dataset) - train_size
train_set, valid_set = torch.utils.data.random_split(train_dataset,
                                                     [train_size, valid_size])

train_gen = torch.utils.data.DataLoader(dataset=train_set,
                                      batch_size = BATCH_SIZE,
                                      shuffle = True)

valid_gen = torch.utils.data.DataLoader(dataset = valid_set,
                                      batch_size = valid_size, 
                                      shuffle = False)


print('Loaded ', len(train_set), 'training images and ', len(valid_set), ' validation images.')
print('Classes detected: ', train_dataset.classes)

In [ ]:
import matplotlib.pyplot as plt
# Show example images after transforms
data_iter = iter(train_gen)
images, labels = next(data_iter)
fig, axes = plt.subplots(figsize = (18,10), ncols=8)
for ii in range(8):
    ax = axes[ii]
    ax.imshow(images[ii].permute(1,2,0))

## Architecture and training
Defining simple net - fully connected network with 2 layers.

In [ ]:
import pytorch_lightning as pl
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report


class ConvNetwork(pl.LightningModule):
    def __init__(self):
      super(ConvNetwork,self).__init__()
      self.conv1 = nn.Conv2d(3, 16, kernel_size=5) # convolutional layer
      self.relu = nn.ReLU() # activation function: relu
      self.conv2 = nn.Conv2d(16, 32, kernel_size=5) # second convolutional layer
      self.fc = nn.Linear(32*20*20, 4) # output layer

      self.loss_function = nn.CrossEntropyLoss()

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = x.view(-1, 32*20*20)
        out = self.fc(x)
        return out
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=lr)
    
    def training_step(self, batch, batch_idx):
        images,labels = batch
        outputs = self(images)
        loss = self.loss_function(outputs, labels)
        self.log('loss', loss) # add loss to track
        return loss

## Training

In [ ]:
net = ConvNetwork()
trainer = pl.Trainer(max_epochs=NUM_EPOCHS,log_every_n_steps=1)
trainer.fit(net,train_gen, valid_gen)

# Testing

In [ ]:
# Add evaluation here